In [1]:
import random
import numpy as np


In [2]:
# Define the graph as an adjacency matrix
graph = [
    [0, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [0, 1, 1, 0]
]

num_vertices = len(graph)
num_colors = 3
num_ants = 4
max_iterations = 100

In [3]:
class Ant:
    def __init__(self, position):
        self.position = position

def fitness(position):
    conflicts = 0
    for i in range(num_vertices):
        for j in range(num_vertices):
            if graph[i][j] and position[i] == position[j]:
                conflicts += 1
    return conflicts

def initialize_ants():
    ants = []
    for _ in range(num_ants):
        position = np.random.randint(num_colors, size=num_vertices)
        ant = Ant(position)
        ants.append(ant)
    return ants

In [10]:
def update_ant(ant, pheromone, alpha, beta):
    for i in range(num_vertices):
        color_probabilities = []
        for color in range(num_colors):
            if color in ant.position:
                color_probabilities.append(0.0)
            else:
                pheromone_level = pheromone[i][color]
                heuristic_value = calculate_heuristic(ant.position, i, color)
                color_probabilities.append(pheromone_level ** alpha * heuristic_value ** beta)
        
        total_prob = sum(color_probabilities)
        
        if np.isnan(total_prob):  # Check if total_prob is NaN
            available_colors = [color for color in range(num_colors) if color not in ant.position]
            num_available_colors = len(available_colors)
            probabilities = [1 / num_available_colors] * num_available_colors  # Assign equal probabilities to available colors
        else:
            probabilities = [p / total_prob for p in color_probabilities]
        
        chosen_color = np.random.choice(range(num_colors), p=probabilities)
        ant.position[i] = chosen_color

def calculate_heuristic(position, vertex, color):
    conflicts = 0
    for i in range(num_vertices):
        if graph[vertex][i] and position[i] == color:
            conflicts += 1
    
    if conflicts == 0:
        return float('inf')  # Assign a very high heuristic value for conflict-free colorings
    
    return 1 / conflicts

In [11]:
def update_pheromone(pheromone, ants, evaporation_rate, Q):
    pheromone *= evaporation_rate
    for ant in ants:
        fitness_value = fitness(ant.position)
        delta_pheromone = Q / fitness_value
        for i in range(num_vertices):
            color = ant.position[i]
            pheromone[i][color] += delta_pheromone

def aco(graph, num_colors, num_ants, max_iterations, alpha, beta, evaporation_rate, Q):
    pheromone = np.ones((num_vertices, num_colors))
    best_position = None
    best_fitness = float('inf')
    
    for _ in range(max_iterations):
        ants = initialize_ants()
        for ant in ants:
            update_ant(ant, pheromone, alpha, beta)
        
        update_pheromone(pheromone, ants, evaporation_rate, Q)
        
        for ant in ants:
            ant_fitness = fitness(ant.position)
            if ant_fitness < best_fitness:
                best_position = np.copy(ant.position)
                best_fitness = ant_fitness
    
    return best_position

In [12]:
# Parameters for ACO
alpha = 1.0  # Pheromone importance
beta = 1.0  # Heuristic importance
evaporation_rate = 0.5  # Evaporation rate
Q = 1.0  # Pheromone deposit constant

# Solve the graph coloring problem using ACO
best_coloring = aco(graph, num_colors, num_ants, max_iterations, alpha, beta, evaporation_rate, Q)

# Print the best color assignments for each vertex
print("Best color assignments:", best_coloring)

ZeroDivisionError: float division by zero